In [1]:
# https://medium.com/@iryna230520/dynamic-few-shot-prompting-overcoming-context-limit-for-chatgpt-text-classification-2f70c3bd86f9

In [2]:
# USE_SAME_FEW_SHOTS=True

In [1]:
number_of_shots = 3

In [2]:
temp = 0.0
num_labels = 17
model_name="gpt-3.5-turbo-0301"
# model_name="gpt-3.5-turbo-instruct"
# model_name="gpt-4-0613"

In [3]:
output_dir = "./data/output/2023.11.28-1_GPT_few-shot-learning_all-17-sdg_1020-texts_GPT3.5_3-shots_SEMANTIC-SIMILARITY/"

final_xlsx_filename = output_dir+f"gpt_1000_texts_{num_labels}sdg_temperature_{temp}_model_name_{model_name}.xlsx"
print(final_xlsx_filename)

classification_report_filename = output_dir+f"gpt_1000_texts_{num_labels}sdg_temperature_{temp}_model_name_{model_name}_classification_report.xlsx"
print(classification_report_filename)

final_xlsx_filename_corrected = output_dir+f"gpt_1000_texts_{num_labels}sdg_temperature_{temp}_model_name_{model_name}_corrected.xlsx"
print(final_xlsx_filename_corrected)

./data/output/2023.11.28-1_GPT_few-shot-learning_all-17-sdg_1020-texts_GPT3.5_3-shots_SEMANTIC-SIMILARITY/gpt_1000_texts_17sdg_temperature_0.0_model_name_gpt-3.5-turbo-0301.xlsx
./data/output/2023.11.28-1_GPT_few-shot-learning_all-17-sdg_1020-texts_GPT3.5_3-shots_SEMANTIC-SIMILARITY/gpt_1000_texts_17sdg_temperature_0.0_model_name_gpt-3.5-turbo-0301_classification_report.xlsx
./data/output/2023.11.28-1_GPT_few-shot-learning_all-17-sdg_1020-texts_GPT3.5_3-shots_SEMANTIC-SIMILARITY/gpt_1000_texts_17sdg_temperature_0.0_model_name_gpt-3.5-turbo-0301_corrected.xlsx


In [6]:
!pip install -q openai --progress-bar off

In [7]:
!pip install -q langchain --progress-bar off

In [8]:
import os
from IPython.display import clear_output
os.environ["OPENAI_API_KEY"] = input("INSERT YOU OPENAI API KEY:")
clear_output()

In [9]:
import pickle
import pandas as pd
import glob

import datetime
from langchain import PromptTemplate
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate

import os
import json
import ast
import re

from typing import Union
from IPython.display import display, HTML

from sklearn.metrics import classification_report

In [10]:
def perform_classification(text: str, chain: LLMChain) -> str:
    """Perform classification using LangChain model API.

    Parameters
    ----------
    text : str
        Input text.
    chain : LLMChain
        LangChain model.

    Returns
    -------
    str or None
        API Response
    """
    try:
        return chain.run(text=text)
    except Exception as e:
        print(e)
        return None
    
def convert_to_dict(row: str) -> dict:
    try:
        row = row.replace("false", "False")
        row = row.replace("true", "True")
        return ast.literal_eval(row)
    except Exception as e:
        print(e, row)
        return row

In [11]:
llm = ChatOpenAI(temperature=temp, model_name=model_name, max_tokens=500)

# Caricamento dati e preparazione prompt


In [10]:
# IN QUESTO ESPERIMENTO CONSIDERO LO STESSO DATASET UTILIZZATO IN 
# GPT_zero-shot_learning/2023.10.17-1_GPT_zero-shot-learning_all-17-sdg.ipynb
# IN MODO CHE I DATI SIANO GLI STESSI 

data = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_1020_texts_TEST.xlsx").drop(columns=['Unnamed: 0'])
# data.index = data.text_id
# data.text_id = data.index
# data = data.drop(columns=['Unnamed: 0', 'goal_label', 'motivation_for_the_answer', 'goal_label_int', 'sdg_probability'])
data

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement
0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN
1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN
2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN
3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN
4,2143,NaN,By Dipo Olowookere\n\nThe President of the Man...,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0
1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0
1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0
1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0


In [13]:
data.shape

(1020, 7)

In [14]:
data.groupby('sdg').count()['text']

sdg
0     60
1     60
2     60
3     60
4     60
5     60
6     60
7     60
8     60
9     60
10    60
11    60
12    60
13    60
14    60
15    60
16    60
Name: text, dtype: int64

In [15]:
top7_train_data_sim_to_each_test_data_filename = "../data/input/top7_train_data_sim_to_each_test_data.pickle"

with open(top7_train_data_sim_to_each_test_data_filename, 'rb') as f:
    top7_train_data_sim_to_each_test_data = pickle.load(f)

In [16]:
data_sel_4_few_shot_examples = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN.xlsx")

### Estraggo i dati per i few-shot

In [17]:
# # TESTI ESTRATTI NEL FEW SHOT 2023.11.21-1_GPT_few-shot-learning_all-17-sdg_1020-texts_GPT3.5_3-shots

# print("""
#       text_id                   doi  \
# 3936     5614  10.2139/SSRN.1406042   
# 1456     2089  10.18356/ee52a573-en   
# 3974     5665  10.18356/3fe10a08-en   

#                                                    text  sdg  labels_negative  \
# 3936  The modern definitions of the inchoate offence...   16              0.0   
# 1456  This refers to the manufacture, use and dispos...   12              3.0   
# 3974  As a result, both the trade and current accoun...    8              0.0   

#       labels_positive  agreement  
# 3936              6.0     1.0000  
# 1456             29.0     0.8125  
# 3974              5.0     1.0000
# """)

In [18]:
# if USE_SAME_FEW_SHOTS==True:
#     data_sel_4_few_shot_examples = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN.xlsx").drop(columns=['Unnamed: 0'])
#     data_sel_4_few_shot_examples = data_sel_4_few_shot_examples[
#         (data_sel_4_few_shot_examples.text_id==5614) |
#         (data_sel_4_few_shot_examples.text_id==2089) |
#         (data_sel_4_few_shot_examples.text_id==5665) |
#         (data_sel_4_few_shot_examples.text_id==370)  |
#         (data_sel_4_few_shot_examples.text_id==1422) |
#         (data_sel_4_few_shot_examples.text_id==4329) |
#         (data_sel_4_few_shot_examples.text_id==4929)
#     ]
# else:
#     data_sel_4_few_shot_examples = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN.xlsx").drop(columns=['Unnamed: 0']).sample(5)

# data_sel_4_few_shot_examples

In [19]:
# few_shots = ""

# for _, row in data_sel_4_few_shot_examples[['text', 'sdg']].iterrows():
#     input_text = "Input text: \""+row['text']+"\"\n"
#     output_text = "Output: \"SDG "+str(row['sdg'])+'\"\n\n'
    
#     few_shots += input_text
#     few_shots += output_text
    
# print(few_shots)

In [20]:
# https://www.un.org/en/exhibits/page/sdgs-17-goals-transform-world#:~:text=GOAL%201%3A%20No%20Poverty%20GOAL,10%3A%20Reduced%20Inequalities%20GOAL%2011

# template = """
# Classify the following text between triple quotes according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015, choosing one (and only one) of the SDG labels based on it best representing the content of the entire text enclosed in triple quotes.

# In the following lines, a text enclosed in triple quotes will be given to you.

# The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

# The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

# If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "SDG 0".

# The following are """+number_of_shots+""" examples of expected classification given an input text:

# """+few_shots+"""


# The text to be processed is:

# ```
# {text}
# ```
# """

In [21]:
# prompt = PromptTemplate(
#     input_variables=["text"],
#     template=template,
# )

In [22]:
# chain = LLMChain(llm=llm, prompt=prompt)

In [11]:
# data_sel = data_sel_4_few_shot_examples
data_sel = data
# data_sel = data[data.sdg==0].sample(10)
# data_sel = data.sample(3)
# data_sel = data.iloc[356:]

data_sel

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement
0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN
1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN
2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN
3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN
4,2143,NaN,By Dipo Olowookere\n\nThe President of the Man...,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0
1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0
1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0
1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0


In [24]:
# data_sel.iloc[:11]

In [25]:
# text_DEBUG = data_sel.iloc[10].text
# text_DEBUG

In [26]:
# perform_classification(text_DEBUG, chain)

In [27]:
# # results = data_sel.apply(lambda x: perform_classication(x["text"], chain), axis=1)
# import time

# cnt = 0
# # cnt = 355
# results = []
# retry = True

# for index, row in data_sel.iterrows():
#     time_sleep=10 # resetto il tempo se non ho avuto problemi con il TPM limit
#     while retry==True:
#         try:
#             time.sleep(time_sleep)
#             time_sleep+=10 # aumento il tempo a ogni tentativo se ho superato il TPM limit
#             result = perform_classification(row["text"], chain)
#             results.append((row["text_id"], result))
#             cnt+=1
#             print(cnt, (row["text_id"], result), time_sleep)
#             retry=False
#         except:
#             retry=True
#             print(cnt, (row["text_id"], result), time_sleep)
    

In [28]:
output_data_path = output_dir

In [34]:
data_sel = data.iloc[650:]

In [35]:
# results = data_sel.apply(lambda x: perform_classication(x["text"], chain), axis=1)
import time

# cnt = 0
cnt = 650
results = []
for index, row in data_sel.iterrows():

    #print("\n\n##################################################")

    #print("EXPECTED_SDG:", row["sdg"], "TEXT:", row["text"].replace('\n',''),'\n\n')

    few_shots_list = top7_train_data_sim_to_each_test_data[row['text_id']][:number_of_shots]

    few_shots = ""

    for train_data_id in few_shots_list:

        # print("##################################################")
        # print(train_data_id)

        train_row = data_sel_4_few_shot_examples[data_sel_4_few_shot_examples.text_id==train_data_id]

        # print(train_row)

        # print(train_row.text.values[0])
        # print(train_row.sdg.values[0])

        input_text = "Input text: \""+train_row.text.values[0].replace('\n','')+"\"\n"
        output_text = "Output: \"---SDG "+str(train_row.sdg.values[0])+'---\"\n\n'

        few_shots += input_text
        few_shots += output_text

    #print(few_shots)
    
    template = """
Classify the following text between triple quotes according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015, choosing one (and only one) of the SDG labels based on it best representing the content of the entire text enclosed in triple quotes.

In the following lines, a text enclosed in triple quotes will be given to you.

The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "SDG 0".

The following are """+str(number_of_shots)+""" examples of expected classification given an input text:

"""+few_shots+"""


The text to be processed is:

```
{text}
```
"""
    try:
        prompt = PromptTemplate(
        input_variables=["text"],
        template=template,
        )

        chain = LLMChain(llm=llm, prompt=prompt)

        result = perform_classification(row["text"], chain)
    except Exception as e:
        result = str(e)                
        
    result_extended = (cnt, row["text_id"], row["sdg"], result)
    with open(output_data_path+f'/sdg_classification_GPT4_{cnt}.pkl', 'wb') as f:
        pickle.dump(result_extended, f)
    
    results.append(result_extended)
    cnt+=1
    #time.sleep(12) # per GPT4
    time.sleep(1)
    print(result_extended)
    

(650, 3782, 10, '---SDG 10---')
(651, 5986, 10, "1 validation error for PromptTemplate\n__root__\n  unmatched '{' in format spec (type=value_error)")
(652, 1908, 10, '---SDG 10---')
(653, 3420, 10, '---SDG 10---')
(654, 2233, 10, '---SDG 8---')
(655, 6504, 10, '---SDG 10---')
(656, 4945, 10, '---SDG 10---')
(657, 1329, 10, '---SDG 8---')
(658, 6782, 10, '---SDG 10---')
(659, 406, 10, '---SDG 8---')
(660, 1968, 11, '---SDG 11---')
(661, 1902, 11, '---SDG 11---')
(662, 6209, 11, '---SDG 11---')
(663, 2986, 11, '---SDG 11---')
(664, 3871, 11, '---SDG 8---')
(665, 1564, 11, '---SDG 11---')
(666, 4187, 11, '---SDG 11---')
(667, 4353, 11, '---SDG 11---')
(668, 4349, 11, '---SDG 11---')
(669, 3479, 11, '---SDG 11---')
(670, 1989, 11, '---SDG 11---')
(671, 739, 11, '---SDG 11---')
(672, 1381, 11, '---SDG 9---')
(673, 3418, 11, '---SDG 11---')
(674, 6481, 11, '---SDG 9---')
(675, 3269, 11, '---SDG 11---')
(676, 6077, 11, '---SDG 11---')
(677, 795, 11, '---SDG 11---')
(678, 3019, 11, '---SDG 11-

(901, 6606, 15, '---SDG 13---')
(902, 4075, 15, '---SDG 15---')
(903, 3920, 15, '---SDG 15---')
(904, 794, 15, '---SDG 15---')
(905, 78, 15, '---SDG 15---')
(906, 2788, 15, '---SDG 15---')
(907, 3261, 15, '---SDG 13---')
(908, 3358, 15, '---SDG 15---')
(909, 759, 15, '---SDG 15---')
(910, 1455, 15, '---SDG 15---')
(911, 724, 15, '---SDG 15---')
(912, 1167, 15, '---SDG 15---')
(913, 5970, 15, '---SDG 15---')
(914, 4268, 15, '---SDG 15---')
(915, 6298, 15, '---SDG 15---')
(916, 588, 15, '---SDG 15---')
(917, 2246, 15, '---SDG 15---')
(918, 1924, 15, '---SDG 13---')
(919, 3281, 15, '---SDG 15---')
(920, 1767, 15, '---SDG 15---')
(921, 1957, 15, '---SDG 13---')
(922, 980, 15, '---SDG 15---')
(923, 1299, 15, '---SDG 15---')
(924, 5980, 15, '---SDG 15---')
(925, 2985, 15, '---SDG 15---')
(926, 1515, 15, '---SDG 13---')
(927, 6479, 15, '---SDG 15---')
(928, 2296, 15, '---SDG 15---')
(929, 3610, 15, '---SDG 15---')
(930, 152, 15, '---SDG 15---')
(931, 658, 15, '---SDG 15---')
(932, 1672, 15, '

In [ ]:
results

In [ ]:
# len(results)

In [ ]:
# len(results_part1)

In [ ]:
# results_tot = results_part1 + results

In [ ]:
# len(results_tot)

In [ ]:
# results = results_tot

In [ ]:
# results_part1 = results.copy()

In [ ]:
# results_part1

In [12]:
import pickle
import pandas as pd
import glob
from sklearn.metrics import classification_report

In [5]:
output_data_path = output_dir

files = glob.glob(output_data_path + "/*.pkl")

In [6]:
results_complete = []
for file in files:
    res = pd.read_pickle(file)

    # try:
    #     if len(res[3]['choices'])>1:
    #         raise Exception('ERROR: Multilabel detected!')
    #         break
    #     sdg_pred = res[3]['choices'][0]['message']['content']
    #     res4df = (res[0], res[1], res[2], sdg_pred)
    #     print(res4df)
    #     tuple_list.append(res4df)
    # except:
    #     print("#################################################################")
    #     print(res)
    #     print("#################################################################")

    # print(res)
    results_complete.append(res)
    # break

In [7]:
results  =results_complete

In [8]:
len(results)

1020

In [9]:
results_processed = pd.DataFrame(results, columns=['cnt', 'text_id', 'sdg','result'])
results_processed['label_pred'] = results_processed['result'].str.extract(r'SDG (\d+)')
results_processed

,cnt,text_id,sdg,result,label_pred
0,852,5523,14,---SDG 14---,14
1,355,4613,5,---SDG 5---,5
2,466,5293,7,---SDG 7---,7
3,210,562,3,---SDG 3---,3
4,577,2791,9,---SDG 9---,9
...,...,...,...,...,...
1015,61,357,1,---SDG 1---,1
1016,836,2077,13,---SDG 13---,13
1017,333,2109,5,---SDG 5---,5
1018,775,1049,12,---SDG 12---,12


In [ ]:
# results_processed.to_excel("gpt3_few_3_shot_tmp.xlsx")

In [ ]:
# results_processed = results.apply(convert_to_dict).apply(pd.Series).loc[:, ["motivation_for_the_answer", "detected_goals", "selected_goal"]]
# results_processed

In [ ]:
# results_processed2 = results.apply(convert_to_dict).apply(pd.Series).loc[:, ["goal_label", "motivation_for_the_answer", "sdg_probability"]]
# results_processed2

In [ ]:
# results_processed = pd.DataFrame(results, columns=['text_id', 'result'])
# results_processed['label_pred'] = results_processed['result'].str.extract(r'SDG (\d+)')
# results_processed

In [13]:
# final = data_sel.merge(results_processed, left_index=True, right_index=True)
final = data_sel.merge(results_processed.drop(columns=['sdg']), left_on='text_id', right_on='text_id')
# final = data_sel.merge(results_processed2, left_index=True, right_index=True)
# final = final.drop(columns='Unnamed: 0')
final

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,cnt,result,label_pred
0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,0,"---SDG 9--- (Industry, Innovation and Infrastr...",9
1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,1,---SDG 0---,0
2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,2,---SDG 16---,16
3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,3,---SDG 9---,9
4,2143,NaN,By Dipo Olowookere\n\nThe President of the Man...,0,NaN,NaN,NaN,4,---SDG 7---,7
...,...,...,...,...,...,...,...,...,...,...
1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,1015,---SDG 16---,16
1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,1016,---SDG 16---,16
1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,1017,---SDG 16---,16
1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,1018,---SDG 16---,16


In [14]:
final_xlsx_filename

'./data/output/2023.11.28-1_GPT_few-shot-learning_all-17-sdg_1020-texts_GPT3.5_3-shots_SEMANTIC-SIMILARITY/gpt_1000_texts_17sdg_temperature_0.0_model_name_gpt-3.5-turbo-0301.xlsx'

In [15]:
final.to_excel(final_xlsx_filename)

In [16]:
final_corrected = final[~final.label_pred.isna()]

In [17]:
import numpy as np
y_true = final_corrected.sdg 
y_pred = final_corrected.label_pred
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1019
1019
              precision    recall  f1-score   support

           0       1.00      0.40      0.57        60
           1       0.71      0.80      0.75        60
           2       0.96      0.88      0.92        60
           3       0.73      0.98      0.84        60
           4       0.90      0.93      0.92        60
           5       0.90      0.92      0.91        60
           6       0.98      0.93      0.96        60
           7       0.88      0.85      0.86        60
           8       0.59      0.75      0.66        60
           9       0.76      0.80      0.78        60
          10       0.76      0.58      0.65        59
          11       0.93      0.90      0.92        60
          12       0.89      0.93      0.91        60
          13       0.78      0.88      0.83        60
          14       0.95      0.97      0.96        60
          15       0.92      0.90      0.91        60
          16       0.89      0.85      0.87        60
          17     

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [18]:
classification_report_filename

'./data/output/2023.11.28-1_GPT_few-shot-learning_all-17-sdg_1020-texts_GPT3.5_3-shots_SEMANTIC-SIMILARITY/gpt_1000_texts_17sdg_temperature_0.0_model_name_gpt-3.5-turbo-0301_classification_report.xlsx'

In [19]:
report=classification_report(y_true, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df.to_excel(classification_report_filename)

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
